In [45]:
#Configuración inicial del archivo

#Invocando librerías necesarias
import os
import numpy as np
import pandas as pd    #para trabajar con las matrices
import math    #para trabajar con exp
from math import e

#Para cambiar el directorio de trabajo:
#os.chdir('L:\Google Drive\Consultoría\Internexa')
os.chdir('L:\Google Drive\Consultoría\Internexa')
#os.chdir('C:/Users/CJ')


In [46]:
#Datos de entrada

#Parámetros para la disminución en la vida útil por sobrecarga
horas_sobrecarga_dia = 3
perdida_vida_diaria_teorica = 0.000133
horas_vida_util_teoricas = 24 / perdida_vida_diaria_teorica
anos_vida_util_teoricos = horas_vida_util_teoricas / (24*365) 
temperatura_normal_trafo = 110
temperatura_ambiente = 30
probabilidad_deterioro_reubicacion = 0.05

#Parámetros para el cálculo de las pérdidas de transformación
#Transformadores monofásicos
#Pérdidas en vacío
Par_1f_vac_1 = 9.8033
Par_1f_vac_2 = 0.7141
#Pérdidas con carga
Par_1f_car_1 = 0.000063
Par_1f_car_2 = -0.02695
Par_1f_car_3 = 10.657
Par_1f_car_4 = 38.267

#Transformadores trifásicos
#Pérdidas en vacío
Par_3f_vac_11 = 10.514
Par_3f_vac_12 = 0.7486
Par_3f_vac_21 = 13.27
Par_3f_vac_22 = 0.7093
Par_3f_vac_31 = 1.227
Par_3f_vac_32 = 554.59
#Pérdidas con carga
Par_3f_car_11 = -0.0103
Par_3f_car_12 = 13.892
Par_3f_car_13 = 106.65
Par_3f_car_21 = 10.465
Par_3f_car_22 = 537
Par_3f_car_31 = 9.2632
Par_3f_car_31 = 1875.2


In [47]:
#################Lectura de los datos de entrada######################
xls = pd.ExcelFile("Datos_entrada_Internexa_21Jun18.xlsx")   
nodos = xls.parse("m1_nodos",usecols = 'A:M')
inventario = xls.parse("m2_inventario_transformadores", usecols = "A:O")
caractecntrafos = xls.parse("carac_tecn_transf",usecols = "A:C")
vidautil = xls.parse("vida_util",usecols = "A:I")

#Poner en el inventari el Id del nodo que es más fácil de trabajar
inventario = inventario.merge(nodos[['id_nodo','id_nodo_Internexa']], on = 'id_nodo_Internexa',how = 'left')

#Indicar a qué grupo pertenece el transformador, para el cálculo de la pérdida de vida útil
inventario['grupo_pvu'] = 1
inventario.loc[(inventario['fases'] == 1) & (inventario['capacidad_kVA'] > 50), 'grupo_pvu'] = 2
inventario.loc[(inventario['fases'] == 3) & (inventario['capacidad_kVA'] >= 150) & (inventario['capacidad_kVA'] < 500), 'grupo_pvu'] = 2
inventario.loc[(inventario['fases'] == 3) & (inventario['capacidad_kVA'] >= 500), 'grupo_pvu'] = 3

#Indicar a qué grupo pertenece el transformador, para el cálculo de las pérdidas de transformación
inventario['grupo_pt'] = 1
inventario.loc[(inventario['fases'] == 3) & (inventario['capacidad_kVA'] >= 150) & (inventario['capacidad_kVA'] <= 800), 'grupo_pt'] = 2
inventario.loc[(inventario['fases'] == 3) & (inventario['capacidad_kVA'] > 800), 'grupo_pt'] = 3
                
#Calcular la vida útil restante del transformador (en meses)
inventario['vida_util_teorica_anos'] = anos_vida_util_teoricos
inventario['vida_util_restante_meses'] = (inventario['vida_util_teorica_anos'] - inventario['anos_uso']) * 12
inventario.loc[inventario['vida_util_restante_meses'] < 0, 'vida_util_restante_meses'] = 1

#Indicar los grupos de la Vida Útil
vidautil['grupo_pvu'] = 1
vidautil.loc[(vidautil['fases'] == 1) & (vidautil['limite_inferior_potencia_KVA'] >= 50), 'grupo_pvu'] = 2
vidautil.loc[(vidautil['fases'] == 3) & (vidautil['limite_inferior_potencia_KVA'] >= 150) & (vidautil['limite_superior_potencia_KVA'] <= 500),'grupo_pvu'] = 2
vidautil.loc[(vidautil['fases'] == 3) & (vidautil['limite_inferior_potencia_KVA'] >= 500), 'grupo_pvu'] = 3

#Concatenar
caractecntrafos['fasesycapacidad'] = caractecntrafos['fases'].map(str) + "-" + caractecntrafos['capacidad_kVA'].map(str)
vidautil['tempyfasesycargprecygrupo'] = vidautil['T_gradC'].map(str) + "-" + vidautil['fases'].map(str) + "-" + vidautil['carga_precedente'].map(str) + "-" + vidautil['grupo_pvu'].map(str)


In [48]:
##########################################
#  Definición de funciones #
##########################################

#Obtener los parámetros un nodo
def parametros_nodo(id_nodo):
    carga_max_nodo = float(nodos[nodos['id_nodo']==id_nodo]['carga_maxima_kVA'])
    carga_promedio_nodo = float(nodos[nodos['id_nodo']==id_nodo]['carga_promedio_kVA'])    
    costokwh = float(nodos[nodos['id_nodo']==id_nodo]['tarifa_KWh_usuario_predominante']) 
    return (carga_max_nodo,carga_promedio_nodo,costokwh)

#Obtener los parámetros de un transformador
def parametros_trafo(id_trafo):
    cap_trafo = float(inventario[inventario['id_trafo']==id_trafo]['capacidad_kVA'])
    fases  = int(inventario[inventario['id_trafo']==id_trafo]['fases'])
    vida_util_t = int(inventario[inventario['id_trafo']==id_trafo]['vida_util_teorica_anos'])
    nodo_actual = int(inventario[inventario['id_trafo']==id_trafo]['id_nodo'])
    costo_reubicacion  = float(inventario[inventario['id_trafo']==id_trafo]['costo_actividad_reubicacion'])
    vida_util_restante = float(inventario[inventario['id_trafo']==id_trafo]['vida_util_restante_meses'])
    grupo  = int(inventario[inventario['id_trafo']==id_trafo]['grupo_pt'])
    fc = str(fases) + '-' + str(cap_trafo)
    costo_nuevo = int(caractecntrafos[caractecntrafos['fasesycapacidad']==fc]['costo_a_nuevo'])
    return (cap_trafo,fases,vida_util_t,nodo_actual,costo_reubicacion,vida_util_restante,grupo,fc,costo_nuevo)

#Costo de las pérdidas de transformación de un par nodo*trafo
def costo_pertransf(id_nodo,id_trafo):
    if id_nodo == 999999:
        costo_perdidas_transformacion = 0
    else:
        #Hallar los parámetros del nodo y del trafo
        carga_max_nodo,carga_promedio_nodo,costokwh = parametros_nodo(id_nodo)
        cap_trafo,fases,vida_util_t,nodo_actual,costo_reubicacion,vida_util_restante,grupo,fc,costo_nuevo = parametros_trafo(id_trafo)
        factor_utilizacion = carga_max_nodo / cap_trafo
            
        #Calcular las pérdidas en vacío y con carga en función de las fases y el grupo en la fase
        if fases == 1:
            perdidas_nom_vacio = Par_1f_vac_1 * cap_trafo ** Par_1f_vac_2
            perdidas_nom_carga = Par_1f_car_1 * cap_trafo ** 3 + Par_1f_car_2 * cap_trafo ** 2 + Par_1f_car_3 * cap_trafo + Par_1f_car_4
        else:
            if grupo == 1:
                perdidas_nom_vacio = Par_3f_vac_11 * cap_trafo ** Par_3f_vac_12
                perdidas_nom_carga = Par_3f_car_11 * cap_trafo ** 2 + Par_3f_car_12 * cap_trafo + Par_3f_car_13
            if grupo == 2:
                perdidas_nom_vacio = Par_3f_vac_21 * cap_trafo ** Par_3f_vac_22
                perdidas_nom_carga = Par_3f_car_21 * cap_trafo + Par_3f_car_22
            if grupo == 3:
                perdidas_nom_vacio = Par_3f_vac_31 * cap_trafo ** Par_3f_vac_32
                perdidas_nom_carga = Par_3f_car_31 * cap_trafo + Par_3f_car_32
    
        #Calcular las pérdidas en hierro y cobre (unidades de las pérdidas: W) 
        perdidas_hierro_W = perdidas_nom_vacio
        perdidas_cobre_W = perdidas_nom_carga * factor_utilizacion ** 2
        perdidas_transformacion_W = perdidas_hierro_W + perdidas_cobre_W
    
        #Monetización de las pérdidas
        costo_perdidas_transformacion = perdidas_transformacion_W / 1000 * costokwh * 24 * 30 * vida_util_restante
    return costo_perdidas_transformacion


#Costo de las pérdidas de vida útil
def costo_pervutil(id_nodo,id_trafo):
    if id_nodo == 999999:
        costo_perdidas_vida_util = 0
    else:
        #Hallar los parámetros del nodo y del trafo
        carga_max_nodo,carga_promedio_nodo,costokwh = parametros_nodo(id_nodo)
        cap_trafo,fases,vida_util_t,nodo_actual,costo_reubicacion,vida_util_restante,grupo,fc,costo_nuevo = parametros_trafo(id_trafo)
    
        #Cálculo de la carga precedente
        carga_precedente_original = carga_promedio_nodo / cap_trafo

        #Aproximación a los valores de la norma
        if carga_precedente_original < ((0.5 + 0.75) / 2):
            carga_precedente = 0.5
        else:
            if carga_precedente_original < ((0.75 + 0.9) / 2):
                carga_precedente = 0.75
            else:
                carga_precedente = 0.9
        
        #Key de búsquedas en la tabla de Vida Útil
        key = str(temperatura_ambiente) + '-' + str(fases) + '-' + str(carga_precedente) + '-' + str(grupo)
    
        #Cálculo de la carga pico o factor de utilizacion
        factor_utilizacion = carga_max_nodo / cap_trafo

        #Calcular el porcentaje diario de pérdida de vida útil real en porcentaje
        theta = temperatura_PC(factor_utilizacion,key)
        factor_envejecimiento = (horas_sobrecarga_dia / 24) * (e**(15000/383 - 15000/(theta + 273))-1)
        perdida_vida_diaria_real = perdida_vida_diaria_teorica * (1 + factor_envejecimiento)
    
        #Valoración de la pérdida de vida útil restante (en pesos, durante lo que queda de vida útil de ese nodo en ese transformador)
        costo_perdidas_vida_util = vida_util_restante * 30 * costo_nuevo * perdida_vida_diaria_real
    return costo_perdidas_vida_util


#Calcular la temperatura del punto más caliente 
def temperatura_PC(factor_utilizacion,key):
    subtabla_vidautil = vidautil.loc[(vidautil['tempyfasesycargprecygrupo']==str(key)) & (vidautil['duracion_pico_carga_h']<=horas_sobrecarga_dia)]
    subtabla_vidautil = subtabla_vidautil.sort_values(['carga_porc'],ascending=[True])
    ncargas = subtabla_vidautil.shape[0]
    carga=factor_utilizacion * 100

    theta = 0
    for i in range (0,ncargas):
        #Primer caso: la carga es muy poca para arrastrar una temperatura, o arrastra justo la de la tabla
        if i == 0:
            if carga < subtabla_vidautil.iloc[i,6]:
                theta = temperatura_normal_trafo
                break
            if carga == subtabla_vidautil.iloc[i,6]:
                theta = subtabla_vidautil.iloc[i,7]
                break            
        #Otros casos: la carga sí es suficiente para arrastrar una temperatura
        else:
            if carga <= subtabla_vidautil.iloc[i,6]:
                theta = subtabla_vidautil.iloc[i,7]
                break
        #Si la temperatura excedía el valor máximo de la tabla, poner el valor máximo de la tabla
        if i == (ncargas - 1) and theta == 0:
            theta = subtabla_vidautil.iloc[i,7]
    return theta 


#Calcular los costos de la actividad de reubicación
def costo_permutacion(id_trafo):
    costo_perm = float(inventario[inventario['id_trafo']==id_trafo]['costo_actividad_reubicacion'])
    return costo_perm

#Calcular los costos del deterioro por reubicación
def costo_deterioro(id_trafo):
    cap_trafo,fases,vida_util_t,nodo_actual,costo_reubicacion,vida_util_restante,grupo,fc,costo_nuevo = parametros_trafo(id_trafo)
    costo_deter = costo_nuevo * probabilidad_deterioro_reubicacion
    return costo_deter

#Encontrar las dos parejas para hacer permutación. Una pareja es un par (nodo*trafo)
#Si debe bloquear un nodo porque no encuentra pareja para él, debe repetir con el siguiente nodo más costoso (por eso el while)
def hallar_parejas_permutacion (solucion):
    id_trafo2 = 0
    while (id_trafo2 ==0):       
        #Identificar el nodo más costoso y su trafo para intercambio
        id_nodo1, id_trafo1 = nodo_mas_costoso (solucion)            
        id_nodo2, id_trafo2 = trafo_mas_costoso (id_nodo1,solucion)
        
        #Si no encuentra un trafo adecuado para el cambio, se bloquea el nodo
        if id_trafo2 == 0:
            solucion.at[solucion['id_nodo']==id_nodo1,'bloqueado']=1
    return (id_nodo1, id_trafo1, id_nodo2, id_trafo2)


#Elegir el nodo más costoso aún no evaluado para permutación
def nodo_mas_costoso (solucion):
    copia_solucion = solucion.copy()
    #Seleccionar los nodos potenciales
    copia_solucion = copia_solucion.loc[(solucion['evaluado'] == 0)]
    copia_solucion = copia_solucion.loc[(solucion['permutado']== 0)]
    copia_solucion = copia_solucion.loc[(solucion['bloqueado']== 0)]
    copia_solucion = copia_solucion.loc[(solucion['costo_operacion'] > 0)]
    #Seleccionar el más costoso de los nodos
    id_nodo1=0
    id_trafo1=0
    if copia_solucion.shape[0] > 0:
        copia_solucion = copia_solucion.sort_values(['costo_operacion'],ascending=[False])
        id_nodo1 = int(copia_solucion.iloc[0,0])
        id_trafo1 = int(copia_solucion.iloc[0,1])
    return (id_nodo1, id_trafo1)


#Elegir el trafo más costoso para hacer permutación con un cierto nodo
def trafo_mas_costoso (id_nodo1,solucion):    
    carga_max_nodo1,carga_promedio_nodo,costokwh_nodo = parametros_nodo(id_nodo1)
    id_trafo1 = int(solucion[solucion['id_nodo']==id_nodo1]['id_trafo'])
    cap_trafo1,fases,vida_util_t,nodo_actual,costo_reubicacion,vida_util_restante,grupo,fc,costo_nuevo = parametros_trafo(id_trafo1)
    
    copia_solucion = solucion.copy()
    #Seleccionar los transformadores potenciales
    copia_solucion = copia_solucion.loc[(solucion['capacidad_kVA']>= carga_max_nodo1)]
    copia_solucion = copia_solucion.loc[(solucion['carga_maxima_kVA']<= cap_trafo1)]
    copia_solucion = copia_solucion.loc[(solucion['permutado']== 0)]
    copia_solucion = copia_solucion.loc[(solucion['bloqueado']== 0)]
    copia_solucion = copia_solucion.loc[(solucion['id_nodo']!= id_nodo1)]
    #Restricción de cargabilidad mínima de un trafo por auditoría de la CREG: 40%
    copia_solucion = copia_solucion.loc[(solucion['capacidad_kVA']<= carga_max_nodo1 / 0.4)]
    
    #Seleccionar el más costoso de los nodos de los trafos
    id_nodo2=0
    id_trafo2=0
    if copia_solucion.shape[0] > 0:
        copia_solucion = copia_solucion.sort_values(['costo_operacion'],ascending=[False])
        id_nodo2 = int(copia_solucion.iloc[0,0])
        id_trafo2 = int(copia_solucion.iloc[0,1])
    return (id_nodo2, id_trafo2)


#Calcular los costos de permutación y deterioro de una solución provisional, respecto a una solución original
def calcular_costos_permutacion(id_trafo1,id_trafo2,solucion_original,solucion_provisional):
    c_permut1 = 0
    c_permut2 = 0
    #Para el trafo1
    id_nodo1_original = int(solucion_original.loc[(solucion_original['id_trafo'] == id_trafo1)]['id_nodo'])
    id_nodo1_solucion = int(solucion_provisional.loc[(solucion_provisional['id_trafo'] == id_trafo1)]['id_nodo'])
    if id_nodo1_original != id_nodo1_solucion:
        c_permut1 = costo_permutacion(id_trafo1)
        c_deter1 = costo_deterioro(id_trafo1)
    #Para el trafo2
    id_nodo2_original = int(solucion_original.loc[(solucion_original['id_trafo'] == id_trafo2)]['id_nodo'])
    id_nodo2_solucion = int(solucion_provisional.loc[(solucion_provisional['id_trafo'] == id_trafo2)]['id_nodo'])
    if id_nodo2_original != id_nodo2_solucion:
        c_permut2 = costo_permutacion(id_trafo2)
        c_deter2 = costo_deterioro(id_trafo2)
    return (c_permut1,c_deter1,c_permut2,c_deter2)


#Verificar si se ha cumplido la condición de parada de las permutaciones
def condicion_de_parada(solucion):
    condicion_parada = 0
    copia_solucion = solucion.copy()
    #Seleccionar los nodos potenciales
    copia_solucion = copia_solucion.loc[(solucion['evaluado'] == 0)]
    copia_solucion = copia_solucion.loc[(solucion['permutado']== 0)]
    copia_solucion = copia_solucion.loc[(solucion['bloqueado']== 0)]
    copia_solucion = copia_solucion.loc[(solucion['costo_operacion'] > 0)]
    if copia_solucion.shape[0] == 0: condicion_parada = 1
    return condicion_parada


In [49]:
######################################################
# Función para evaluar el costo de la solución actual#
######################################################

def calculos_solucion_original():
    nnodos = nodos.shape[0]
    ntrafos = inventario.shape[0]
    solucion_original = pd.DataFrame(columns=['id_nodo','id_trafo','costo_pertec','costo_vidautil','costo_operacion','costo_perm','costo_deterioro','evaluado','permutado','bloqueado'])
    #print(inventario)

    for i in range (0,ntrafos):
        id_trafo = int(inventario.loc[i,'id_trafo'])
        id_nodo = int(inventario.loc[i,'id_nodo'])
                  
        #Cálculos de costos para los nodos (reales y en bodega)
        if id_nodo != 999999: 
            cpt = costo_pertransf(id_nodo,id_trafo)
            cpvu = costo_pervutil (id_nodo,id_trafo)
        else:
            cpt = 0
            cpvu = 0
        c_operacion = cpt + cpvu               
        #almacenar el costo para ese arreglo en particular
        solucion_original.loc[i]= [id_nodo,id_trafo,cpt,cpvu,c_operacion,0,0,0,0,0]

    #Poner carga máxima de nodos y capacidades de transformadores en la solución original
    solucion_original = solucion_original.merge(inventario[['id_trafo','capacidad_kVA']], on = 'id_trafo',how = 'left')
    solucion_original = solucion_original.merge(nodos[['id_nodo','carga_maxima_kVA']], on = 'id_nodo',how = 'left')

    #Almacenar la solución original y calcular su costo
    solucion_original.to_csv('solucion_original.csv')
    costo_solucion_original = solucion_original['costo_operacion'].sum()
    
    return (solucion_original,costo_solucion_original)

In [50]:
######################################
# Función para hacer una permutación #
######################################

def permutar(solucion):
    #Elegir el nodo más costoso y el trafo más costoso que se adecua a él
    id_nodo1, id_trafo1, id_nodo2, id_trafo2 = hallar_parejas_permutacion (solucion)
    solucion.at[solucion['id_nodo']==id_nodo1,'evaluado'] = 1

    #Arrastrar los costos de operación actuales en los dos nodos
    c_operacion_actual_1 = 0
    c_operacion_actual_2 = 0
    if id_nodo1 != 999999: c_operacion_actual_1 = float(solucion[solucion['id_nodo']==id_nodo1]['costo_operacion'])
    if id_nodo2 != 999999: c_operacion_actual_2 = float(solucion[solucion['id_nodo']==id_nodo2]['costo_operacion'])
    c_operacion_actual = c_operacion_actual_1 + c_operacion_actual_2

    #Calcular los costos de operación después de la permutación en los dos nodos
    cpt1 = costo_pertransf(id_nodo1,id_trafo2)
    cpvu1 = costo_pervutil (id_nodo1,id_trafo2)
    cpt2 = costo_pertransf(id_nodo2,id_trafo1)
    cpvu2 = costo_pervutil (id_nodo2,id_trafo1)
    c_operacion_permutacion_1 = cpt1 + cpvu1 
    c_operacion_permutacion_2 = cpt2 + cpvu2 
    c_operacion_permutacion = c_operacion_permutacion_1 + c_operacion_permutacion_2

    #Calcular los costos de permutación de los transformadores
    solucion_provisional = solucion.copy()
    solucion_provisional.at[(solucion['id_nodo']==id_nodo1) & (solucion['id_trafo']==id_trafo1),'id_trafo']=id_trafo2
    solucion_provisional.at[(solucion['id_nodo']==id_nodo2) & (solucion['id_trafo']==id_trafo2),'id_trafo']=id_trafo1
    c_permut1,c_deter1,c_permut2,c_deter2 = calcular_costos_permutacion(id_trafo1,id_trafo2,solucion_original,solucion_provisional)
    c_permut = c_permut1 + c_permut2
    c_deter = c_deter1 + c_deter2

    #Hacer el cambio oficial en "solucion"
    permutacion_realizada = 0
    if c_operacion_actual >= (c_operacion_permutacion + c_permut + c_deter):
        permutacion_realizada = 1
        #Almacenar las capacidades
        capacidad_trafo1 = float(solucion[solucion['id_trafo']==id_trafo1]['capacidad_kVA'])
        capacidad_trafo2 = float(solucion[solucion['id_trafo']==id_trafo2]['capacidad_kVA'])
    
        #Actualizar la matriz de solución
        solucion.at[(solucion['id_nodo']==id_nodo1) & (solucion['id_trafo']==id_trafo1),'costo_operacion']=c_operacion_permutacion_1
        solucion.at[(solucion['id_nodo']==id_nodo2) & (solucion['id_trafo']==id_trafo2),'costo_operacion']=c_operacion_permutacion_2
        solucion.at[(solucion['id_nodo']==id_nodo1) & (solucion['id_trafo']==id_trafo1),'costo_perm']=c_permut2
        solucion.at[(solucion['id_nodo']==id_nodo2) & (solucion['id_trafo']==id_trafo2),'costo_perm']=c_permut1
        solucion.at[(solucion['id_nodo']==id_nodo1) & (solucion['id_trafo']==id_trafo1),'costo_deterioro']=c_deter2
        solucion.at[(solucion['id_nodo']==id_nodo2) & (solucion['id_trafo']==id_trafo2),'costo_deterioro']=c_deter1
        solucion.at[(solucion['id_nodo']==id_nodo1) & (solucion['id_trafo']==id_trafo1),'capacidad_kVA']=capacidad_trafo2
        solucion.at[(solucion['id_nodo']==id_nodo2) & (solucion['id_trafo']==id_trafo2),'capacidad_kVA']=capacidad_trafo1
        solucion.at[(solucion['id_nodo']==id_nodo1) & (solucion['id_trafo']==id_trafo1),'permutado']=1
        solucion.at[(solucion['id_nodo']==id_nodo2) & (solucion['id_trafo']==id_trafo2),'permutado']=0
        solucion.at[(solucion['id_nodo']==id_nodo1) & (solucion['id_trafo']==id_trafo1),'id_trafo']=id_trafo2
        solucion.at[(solucion['id_nodo']==id_nodo2) & (solucion['id_trafo']==id_trafo2),'id_trafo']=id_trafo1        
        
    #Guardar estadísticas
    #solucion.to_csv('solucion_final.csv')
    costo_operacion_solucion = solucion['costo_operacion'].sum()
    costo_permutaciones_solucion = solucion['costo_perm'].sum()
    costo_deterioros_solucion = solucion['costo_deterioro'].sum()
    costo_solucion_final = costo_operacion_solucion + costo_permutaciones_solucion + costo_deterioros_solucion

    return (solucion,costo_operacion_solucion,costo_permutaciones_solucion,costo_solucion_final,permutacion_realizada)

In [51]:
#Función para cálculos en bodega
def bodega(solucion_original,solucion_final):
    bodega_original = solucion_original.loc[solucion_original['id_nodo']==999999]
    trafos_en_bodega_original = bodega_original.shape[0]
    capacidad_en_bodega_original = bodega_original['capacidad_kVA'].sum()

    bodega_final = solucion_final.loc[solucion_final['id_nodo']==999999]
    trafos_en_bodega_final = bodega_final.shape[0]
    capacidad_en_bodega_final = bodega_final['capacidad_kVA'].sum()

    return (bodega_original,bodega_final)


In [52]:
########################################
######  Main #######
########################################

#Calcular los costos de la solución actual
solucion_original,costo_solucion_original = calculos_solucion_original()

solucion=solucion_original.copy()

progreso_soluciones = pd.DataFrame(columns=['iteracion','permutaciones','costo_operacion','costo_perm','costo_total'])
progreso_soluciones.loc[0] = [0,0,costo_solucion_original,0,costo_solucion_original]
total_iteraciones = 0
total_permutaciones = 0
condicion_parada = condicion_de_parada (solucion)

while (condicion_parada == 0):
    solucion_final,costo_operacion_solucion,costo_permutaciones_solucion,costo_solucion_final,permutacion_realizada = permutar (solucion)
    total_permutaciones = total_permutaciones + permutacion_realizada
    total_iteraciones = total_iteraciones + 1
    
    #Guardar progreso de la simulación
    progreso_soluciones.loc[total_iteraciones] = [total_iteraciones,total_permutaciones,costo_operacion_solucion,costo_permutaciones_solucion,costo_solucion_final]     

    #Evaluar condición de parada
    condicion_parada = condicion_de_parada (solucion)

#Evaluación de la bodega
bodega_original,bodega_final = bodega(solucion_original, solucion_final)

#Exportar los resultados
solucion.to_csv('solucion_final.csv')
progreso_soluciones.to_csv('progreso_soluciones.csv')
bodega_original.to_csv('bodega_original.csv')
bodega_final.to_csv('bodega_final.csv')
print(costo_solucion_original)
print(costo_solucion_final)
#print(progreso_soluciones)

2721842774.896752
2427134594.458605


In [55]:
#Para pasar este código que está en .ipynb (jupyter) a un .py (Python)
#En la consola de Anaconda, ubicada en la carpeta donde está este archivo, escribir:
jupyter nbconvert --to script Internexa_v50_LCadavid_25Jun18.ipynb


SyntaxError: invalid syntax (<ipython-input-55-04ef329c4dd8>, line 1)